In [2]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Description: the script calculates current density maps starting from electric magnitude
# maps (Emag) generated using ROAST, for each of the 3 datasets included in the analysis
# (namely wp1a, wp1b and wp2a).
# Results are saved in 4d nifti files (1 volume per subject). For convenience, electric 
# field maps and masks are also saved in 4d nifti files.

# Imports
import warnings
warnings.filterwarnings("ignore")
import os
import glob
import numpy as np
from nilearn import image, plotting
from nilearn.image import new_img_like
from scipy import ndimage
import scipy.io
from ipywidgets import IntProgress
from IPython.display import display
import time

# Function to calculate current density
from custom_functions.maps_functions import current_density_efield

In [35]:
## Parameters and variables: 
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' 
output_folder = 'D:\\roast-chapter3\\wp_all_results\\' # where to save e-field maps, current density maps and brain masks
data_folder = 'D:\\roast-chapter3\\'
# Tissue conductivities:
conductivities = [0.126, 0.276, 1.65] #WM, GM and CSF
fwhm_k = 4 # FWHM kernel used to smooth current density maps

# Datasets names and subjects lists
db_names = ['wp2a', 'wp1b']
db_subjects = [['01','02','03','04','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','22','23','24'], #Wp2a
               ['01','02','03','04','05','06','07','08','09','10','11','12','13','15','16','17','18','19','21','22','23'], #Wp1b
               #['03','04','05','07','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26'] # Wp1a (copying data atm)
               ]

## Loading AAL3 atlas and extracting M1 / Thalamus ROIs (regions of interest)
# AAL3 atlas paper: https://www.oxcns.org/papers/607%20Rolls%20Huang%20Lin%20Feng%20Joliot%202020%20AAL3.pdf 
AAl3_path = os.path.join(main_folder, 'rois', 'AAL3v1_1mm.nii')
AAL3_atlas = image.load_img(AAl3_path)

## Creating M1 and Th masks from the AAL3 atlas. Loading MNI template
# AAL3 index for left M1 = 1
m1 = image.math_img("np.where(img == 1, 1, 0)", img = AAL3_atlas) 
# AAL3 index for TH = 121 - 149 (odd values only (left thalamus)) --> I'm not convinced about this one, ask davinia
th = image.math_img("np.where(np.isin(img, np.arange(121, 150, 2)), 1, 0)", img = AAL3_atlas) 

# MNI template for plotting and masking purposes
bg_img_map = image.load_img(os.path.join(main_folder, 'rois', 'MNI152_T1_1mm_Brain.nii'))
bg_img_map_smoothed = image.smooth_img(bg_img_map, fwhm=4) # MNI template smoothed (plotting purposes only)

In [42]:
for db_id, db in enumerate(db_names): #Iterate all datasets
    db_path = os.path.join(data_folder, db)

    for i, subject in enumerate(db_subjects[db_id]): #Iterate all subjects
        path = db_path + '\sub-' + subject # Subject folder

        # Loading normalised Electric field magnitude map (emag) (unit: V/m) (e.g. wsub-*_emag.nii)
        # NB: The map was resampled and normalised with wp*_roast_3_post_roast_preprocessing.m
        emag_map = image.load_img(glob.glob(path + '\w*_emag.nii'))

        # The above scan (and all the other ones that will be loaded) has 4 dimensions, with the 4th one referring to
        # the number of volumes. However, these are 3d scans, I therefore drop the 4th dimension (i.e. from (157, 189, 156, 1)) to
        # (157, 189, 156). This is handy when performing mathematical operations on the scans.
        scan_shape = emag_map.get_fdata().shape[0:3] # Shape of the scan (should be = (157, 189, 156))    
        emag_data = emag_map.get_fdata().reshape(scan_shape) # Data with 4th dimention in the array dropped
        emag_map = new_img_like(emag_map, emag_data) #  Restoring the data into a nibabel object

        # Resampling MNI anatomical file and ROIs to emag map (so that they have the same shape).
        # This is done only once as the shape is the same for every participant's map.
        if not 'mni_resampled' in locals():
            mni_resampled = image.resample_to_img(bg_img_map, emag_map, interpolation = 'nearest')
        if not 'm1_resampled' in locals():
            m1_resampled = image.resample_to_img(m1, emag_map, interpolation = 'nearest')
            th_resampled = image.resample_to_img(th, emag_map, interpolation = 'nearest')

